In [535]:
import re
from collections import Counter
import numpy as np

In [536]:
with open('test.txt') as f:
    input = [[int(c) for c in line.strip('\n')] for line in f]

input = np.asarray(input)

mask = np.ndarray(input.shape,dtype='int')
mask.fill(True)

input

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [537]:
test = input[:, :]

test

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [538]:
mask = np.ndarray(test.shape,dtype='int')
mask.fill(True)
mask

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [539]:
def get_neighbors(map, mask, coord):
    neighbors = []
    dummy = -1
    value, i, j = coord
    #print(f'incoming coord {value=}, {i=} {j=}')

    w = (map[i][j - 1] if j > 0 else dummy, i, j - 1)
    e = (map[i][j + 1] if j < (len(map) - 1) else dummy, i, j + 1)
    s = (map[i + 1][j] if i < (len(map) - 1) else dummy, i + 1, j)
    n = (map[i - 1][j] if i > 0 else dummy, i - 1, j)
    #print(f'incoming coord {n=}, {s=} {e=} {w=} {len(map)=}')


    #nw = (map[i - 1][j - 1] if ((j > 0) & (i > 0)) else dummy, i - 1, j - 1)
    #ne = (map[i + 1][j - 1] if ((j > 0) & (i < len(map) - 1)) else dummy, i + 1, j - 1)
    #sw = (map[i - 1][j + 1] if ((j < len(map) - 1) & (i > 0)) else dummy, i - 1, j + 1)
    #se = (map[i + 1][j + 1] if ((j < len(map) - 1) & (i < len(map) - 1)) else dummy, i + 1, j + 1)

    for n_value, n_i, n_j in [n, s, e, w]:#, nw, ne, sw, se]:

        if (n_value != -1):
            #print(f'for coord {n_value=}, {n_i=} {n_j=} {mask[n_i][n_j]=}')
            if(mask[n_i][n_j]):
                neighbors.append((n_value, n_i, n_j))

    #print('nighbors', neighbors)
    return neighbors

        

    

In [540]:
def get_path_counts(map, mask, coord, proper_end):
    current_value, i, j = coord
    values_masks = []
    mask[i][j] = False
    #print(f'{current_value=} {i=} {j=} \n{mask=} {proper_end=}')
    

    #print(f'final {map.shape[0] - 1=}, {map.shape[1] - 1=} {map.shape=}')
    if (i, j) == (map.shape[0] - 1, map.shape[1] - 1):
        #print('UPLNY KONEC', [current_value], mask)
        proper_end = True
        return [(current_value, mask, proper_end)]
    else:
        counter = 1
        neighbors = get_neighbors(map, mask, coord)

        if len(neighbors) > 0:
            for gn_v, gn_i, gn_j in neighbors:
                #print(f'{counter}. iteration')

                cnts = get_path_counts(map, mask.copy(), (gn_v, gn_i, gn_j), proper_end)
                #values_masks.extend([(current_value + val, mask & msk) for val, msk in cnts])
                
                for val, msk, p_end in cnts:
                    if p_end:
                        new_mask = mask & msk
                        new_cnt = current_value + val
                        #print('mask\n', mask)
                        ##print('msk\n', msk)
                        #mask &= msk
                        #print('mask\n', new_mask)
                        values_masks.append((new_cnt, new_mask, p_end))

                counter += 1

                #print('counts', values_masks)
        else:
            values_masks.append((current_value, mask, False))
        
        return values_masks


In [541]:
test

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [542]:
mask

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [543]:
mask.fill(True)

test[0][0] = 0
paths = get_path_counts(test, mask, (0, 0, 0), False)

KeyboardInterrupt: 

In [544]:
min = paths[0][0]
for score, m, pe in paths:
    if (score < min):
        print(score, pe)
        min = score

min

95 True
91 True
80 True
67 True
65 True
62 True
60 True
55 True
53 True
49 True
47 True
44 True
43 True
30 True
29 True
27 True
24 True


24